In [7]:
# Generating a toy dataset.
# DO NOT MODIFY THIS PART

import numpy as np
import math
import random as rand

paras = list((rand.random() - 0.5 for _ in range(9)))

def y_gen(x):
    h_11 = math.tanh(paras[0] * x[0] + paras[1] * x[1] + paras[2])
    h_12 = math.tanh(paras[3] * x[0] + paras[4] * x[1] + paras[5])
    h_21 = 1/(1 + np.exp(-(paras[6] * h_11 + paras[7] * h_12 + paras[8])))
    return h_21 + ((rand.random()-0.5)/100 if rand.random()>0.6 else 0)

n = 300
x = list(zip((rand.random() - 0.5 for _ in range(n)), (rand.random() - 0.5 for _ in range(n))))
y = list(map(y_gen, x))
y = [(i-min(y))/(max(y)-min(y)) for i in y]

In [8]:
# Spliting dataset into training, validation, and test.
# DO NOT MODIFY THIS PART

from sklearn.model_selection import train_test_split

r = 0.25
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=r)

x_val = x_train[:int(n*r)]
partial_x_train = x_train[int(n*r):]
y_val = y_train[:int(n*r)]
partial_y_train = y_train[int(n*r):]

In [9]:
# DO NOT MODIFY THIS PART

from keras import models
from keras import layers
from keras import optimizers

def create_model(learning_rate=0.01, beta_1=0.9, beta_2=0.999):
    model = models.Sequential()
    model.add(layers.Dense(2, activation='tanh', input_shape=(2,)))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2), loss='mse')
    return model

In [10]:
# Complete the cell to find a best hyper-parameter setting using GridSerachCV

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
model_1=KerasRegressor(build_fn=create_model,nb_epoch=900,batch_size=100)
hyper_parameter_grid = [{
    "learning_rate": [0.1,0.01,0.001,0.011,0.111],
    "beta_1":[0.7,0.8,0.9,0.99,0.6],
    "beta_2":[0.9,0.99,0.999,0.9999,0.99999]
    }]
grid_score = GridSearchCV(model_1,param_grid=hyper_parameter_grid,cv=3,scoring="neg_mean_squared_error")



    

C:\Users\030740227\AppData\Local\Temp\ipykernel_23724\2973705917.py:5: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_1=KerasRegressor(build_fn=create_model,nb_epoch=900,batch_size=100)


In [11]:
grid_score.fit(partial_x_train,partial_y_train)

2/2 [==============================] - 0s 4ms/step - loss: 0.0798


GridSearchCV(cv=3,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000002088A1B2E50>,
             param_grid=[{'beta_1': [0.7, 0.8, 0.9, 0.99, 0.6],
                          'beta_2': [0.9, 0.99, 0.999, 0.9999, 0.99999],
                          'learning_rate': [0.1, 0.01, 0.001, 0.011, 0.111]}],
             scoring='neg_mean_squared_error')

In [12]:
results = grid_score.cv_results_
i = 0
while i < len(results["mean_test_score"]):
    mean_score = results["mean_test_score"][i]
    params = results["params"][i]
    print("%f using %s" % (mean_score, params))
    i += 1
print("Best: %f using %s" % (grid_score.best_score_, grid_score.best_params_))

-0.085587 using {'beta_1': 0.7, 'beta_2': 0.9, 'learning_rate': 0.1}
-0.076795 using {'beta_1': 0.7, 'beta_2': 0.9, 'learning_rate': 0.01}
-0.086524 using {'beta_1': 0.7, 'beta_2': 0.9, 'learning_rate': 0.001}
-0.083892 using {'beta_1': 0.7, 'beta_2': 0.9, 'learning_rate': 0.011}
-0.050377 using {'beta_1': 0.7, 'beta_2': 0.9, 'learning_rate': 0.111}
-0.054679 using {'beta_1': 0.7, 'beta_2': 0.99, 'learning_rate': 0.1}
-0.049449 using {'beta_1': 0.7, 'beta_2': 0.99, 'learning_rate': 0.01}
-0.069904 using {'beta_1': 0.7, 'beta_2': 0.99, 'learning_rate': 0.001}
-0.078123 using {'beta_1': 0.7, 'beta_2': 0.99, 'learning_rate': 0.011}
-0.057792 using {'beta_1': 0.7, 'beta_2': 0.99, 'learning_rate': 0.111}
-0.070187 using {'beta_1': 0.7, 'beta_2': 0.999, 'learning_rate': 0.1}
-0.042287 using {'beta_1': 0.7, 'beta_2': 0.999, 'learning_rate': 0.01}
-0.078357 using {'beta_1': 0.7, 'beta_2': 0.999, 'learning_rate': 0.001}
-0.056206 using {'beta_1': 0.7, 'beta_2': 0.999, 'learning_rate': 0.011}
-0